# Klasyfikacja KNN

W tym notatniku demonstrujemy klasyfikację przy użyciu algorytmu *K-Nearest Neighbors* (KNN)


In [1]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report

# Wczytanie danych z pliku CSV
df = pd.read_csv('Social_Network_Ads.csv')

# Podgląd pierwszych 5 wierszy
df.head()


,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [2]:

# Usuwamy kolumnę 'User ID', która jest identyfikatorem technicznym (nie wpływa na klasyfikację)
df = df.drop('User ID', axis=1)

# Kodowanie kolumny 'Gender' (Female -> 0, Male -> 1)
df['Gender'] = df['Gender'].map({'Female': 0, 'Male': 1})

# Definiowanie cech (X) i etykiety (y)
X = df[['Gender', 'Age', 'EstimatedSalary']]
y = df['Purchased']

# Sprawdzenie wymiarów danych
X.shape, y.shape


((400, 3), (400,))

In [3]:
# Podział danych na część treningową i testową
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Skalowanie cech (standard scaler)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Tworzymy model KNN z 5 sąsiadami
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_scaled, y_train)

# Predykcje na zbiorze testowym
y_pred = knn_model.predict(X_test_scaled)

# Ocena modelu
print("Macierz pomyłek:", confusion_matrix(y_test, y_pred))
print("Raport klasyfikacji:")
print(classification_report(y_test, y_pred))

Macierz pomyłek: [[55  3]
 [ 1 21]]
Raport klasyfikacji:
              precision    recall  f1-score   support

           0       0.98      0.95      0.96        58
           1       0.88      0.95      0.91        22

    accuracy                           0.95        80
   macro avg       0.93      0.95      0.94        80
weighted avg       0.95      0.95      0.95        80



In [4]:

# Ręczne przykłady do sprawdzenia (Gender, Age, EstimatedSalary)
# Przykład 1: Kobieta, 30 lat, 50 000
# Przykład 2: Mężczyzna, 40 lat, 80 000

sample_data = pd.DataFrame({
    'Gender': [0, 1],
    'Age': [30, 40],
    'EstimatedSalary': [50000, 80000]
})

# Skalowanie przykładowych danych
sample_data_scaled = scaler.transform(sample_data)

# Predykcja klasy
sample_predictions = knn_model.predict(sample_data_scaled)

for i, pred in enumerate(sample_predictions):
    osoba = "Kobieta" if sample_data.loc[i, 'Gender'] == 0 else "Mężczyzna"
    print(f"Przykład {i+1}: {osoba}, wiek {sample_data.loc[i, 'Age']} lat, zarobki {sample_data.loc[i, 'EstimatedSalary']} zł -> Kupno: {pred}")


Przykład 1: Kobieta, wiek 30 lat, zarobki 50000 zł -> Kupno: 0
Przykład 2: Mężczyzna, wiek 40 lat, zarobki 80000 zł -> Kupno: 0


In [5]:
# Wyszukiwanie 5 najbliższych sąsiadów dla przykładowej osoby
# Wybieramy przykład pierwszy (kobieta, 30 lat, 50 000)
target = sample_data.iloc[[0]]  # DataFrame z jednym wierszem

# Skalowanie i uzyskanie 5 najbliższych sąsiadów (odległości + indeksy)
target_scaled = scaler.transform(target)

# Zwraca dwie macierze: distances (odległości) i indices (indeksy w zbiorze treningowym)
distances, indices = knn_model.kneighbors(target_scaled, n_neighbors=5)

# Zapamiętujemy listę DataFrame sąsiadów (cechy + etykieta)
# Używamy X_train i y_train, a nie przeskalowanych danych
neighbors = X_train.iloc[indices[0]].copy()
neighbors['Purchased'] = y_train.iloc[indices[0]].values
neighbors['distance'] = distances[0]

print("Pięciu najbliższych sąsiadów (dane, Purchased, distance):")
print(neighbors)

Pięciu najbliższych sąsiadów (dane, Purchased, distance):
     Gender  Age  EstimatedSalary  Purchased  distance
169       0   29            47000          0  0.130401
33        0   28            44000          0  0.260802
41        0   33            51000          0  0.293068
47        0   27            54000          0  0.313814
138       0   28            59000          0  0.325253
